In [2]:
#!pip install --upgrade tensorflow
!pip install Keras
!pip install pandas
#!pip install pandas
!pip install matplotlib


  Using cached https://files.pythonhosted.org/packages/ad/fd/6bfe87920d7f4fd475acd28500a42482b6b84479832bdc0fe9e589a60ceb/Keras-2.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/b1/3367ea1f372957f97a6752ec725b87886e12af1415216feec9067e31df70/numpy-1.16.5-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/90/3216b8f6d69905a320352a9ca6802a8e39fdb1cd93133c3d4163db8d5f19/h5py-2.10.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1d/f6/7c16d60aeb3694e5611976cb4f1eaf1c6b7f1e7c55771d691013405a02ea/scipy-1.2.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached ht

## ENCODE FUNCTION

In [17]:
import base64
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, zero_one_loss
from sklearn.model_selection import train_test_split
import itertools


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

#{name}-{tv}
    
# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l
        
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low


# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

## READ DATASET

In [18]:
from keras.utils.data_utils import get_file
#ver como llamar con hadoop.
try:
    path = './julioredux3.csv'
    #ugr_file = open(path,'r')
    #path = get_file('kddcup.data.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz')
except:
    print('Error loading')
    raise
    
print(path) 

# This file is a CSV, just no CSV extension or headers
# Download from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
df = pd.read_csv(path, header=None)

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'te',
    'td',
    'sa',
    'da',
    'sp',
    'dp',
    'pr',
    'flg',
    'fwd',
    'stos',
    'pkt',
    'byt',
    'norm'
]

# display 5 rows
df[0:6]

./julioredux3.csv
Read 2493050 rows.


,te,td,sa,da,sp,dp,pr,flg,fwd,stos,pkt,byt,norm
0,2016-07-27 13:43:21,48.380,187.96.221.207,42.219.153.7,53,53,UDP,.A....,0,0,2,209,background
1,2016-07-27 13:43:21,48.380,42.219.153.7,187.96.221.207,53,53,UDP,.A....,0,0,2,167,background
2,2016-07-27 13:43:25,50.632,42.219.153.191,62.205.150.146,80,1838,TCP,.AP...,0,0,9,2082,background
3,2016-07-27 13:43:25,51.052,62.205.150.146,42.219.153.191,1838,80,TCP,.AP...,0,0,9,7118,background
4,2016-07-27 13:43:27,46.996,92.225.28.133,42.219.155.111,443,59867,TCP,.AP...,0,0,4,674,background
5,2016-07-27 13:43:27,48.852,143.72.8.137,42.219.154.107,53,60019,UDP,.A....,0,0,2,188,background


## ANALYSING DATASET

In [7]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])
            
def encode_categorical(trainDF,name,df):
        one_hot = pd.get_dummies(df[name])
        one_hot.columns = ["{}-{}".format(name,x) for x in one_hot.columms]
        return trainDF.join(one_hot)
    

In [19]:
df.info()
df.head()
df.drop(['te'], axis=1, inplace=True)

#encode_text_index(df, 'te')
#print ("1")
encode_text_index(df, 'td')
print ("2")
encode_text_index(df, 'sa')
print ("3")
encode_text_index(df, 'da')
print ("4")
encode_numeric_zscore(df, 'sp')
print ("5")
encode_numeric_zscore(df, 'dp')
print ("6")
encode_text_index(df, 'pr')
print ("7")
encode_text_index(df, 'flg')
print ("8")
encode_text_index(df, 'fwd')
print ("9")
encode_numeric_zscore(df, 'stos')
print ("10")
encode_numeric_zscore(df, 'pkt')
print ("11")
encode_numeric_zscore(df, 'byt')
print ("12")
#encode_text_dummy(df, 'norm')
outcomes = encode_text_index(df, 'norm')
num_classes = len(outcomes)

#df.dropna(inplace=True,axis=0)
df.head()
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2493050 entries, 0 to 2493049
Data columns (total 13 columns):
te      object
td      float64
sa      object
da      object
sp      int64
dp      int64
pr      object
flg     object
fwd     int64
stos    int64
pkt     int64
byt     int64
norm    object
dtypes: float64(1), int64(6), object(6)
memory usage: 247.3+ MB
2
3
4
5
6
7
8
9
10
11
12


,td,sa,da,sp,dp,pr,flg,fwd,stos,pkt,byt,norm
0,12076,18992,158583,-0.978729,-0.859107,6,5,0,-0.378942,-0.009183,-0.005546,0
1,12076,54929,71571,-0.978729,-0.859107,6,5,0,-0.378942,-0.009183,-0.005558,0
2,12624,54837,181629,-0.977618,-0.785869,5,13,0,-0.378942,-0.006512,-0.005012,0
3,12724,64385,158463,-0.905268,-0.857999,5,13,0,-0.378942,-0.006512,-0.003574,0
4,11736,79773,158882,-0.962679,1.595049,5,13,0,-0.378942,-0.008420,-0.005413,0


In [28]:
##DECISION TREE MODEL

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

X, y = to_xy(df,'norm')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
decision_tree = tree.DecisionTreeClassifier()
decision_tree = decision_tree.fit(X_train, y_train)
#predict the response for test dataset
pred = decision_tree.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, pred))

Accuracy: 0.9996831190710175


In [30]:
##Vizualizacion Decision Tree

#!pip install graphviz

#!pip install pydotplus
from sklearn import tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import graphviz 

dot_data = tree.export_graphviz(decision_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("UGR") 

dot_data = tree.export_graphviz(decision_tree, out_file=None, 
               feature_names=outcomes.df,  
               #class_names=iris.target_names,  
               filled=True, rounded=True,  
               special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

#tree.plot_tree(decision_tree.fit(X_train, y_train))

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'UGR'], make sure the Graphviz executables are on your systems' PATH